## UROP Project
## Plant Disease Detection Using Image Processing
#### Group:
####    N.Akhila(AP19110010322)
####    K.Manasa(AP19110010343)
####    S.Navya(AP19110010417)
####    K.N.P.Suparnika(AP19110010480)

### Importing the dataset

In [1]:
import cv2
import os
import glob

In [2]:

os.chdir(r"C:\Users\pavan\Downloads\Plant_leave_diseases_dataset_without_augmentation")
all_subdirs = [d for d in os.listdir('.') if os.path.isdir(d)]
#all_subdirs contain the names of leaf category
# folder has the paths to reach the folder. These two will help to extract the images from the folders.
folders=[]
for dirs in all_subdirs:
    dir = os.path.join(r"C:\Users\pavan\Downloads\Plant_leave_diseases_dataset_without_augmentation", dirs)
    os.chdir(dir)
    current = os.getcwd()
    new = str(current).split("/")
    folders.append(new)


In [3]:
#print(all_subdirs)
#print(folders)

In [4]:
all_images=[] #all_images conatins the images in each of the folder
def readimg(s):
    images = [cv2.imread(file) for file in glob.glob(s+"\*.jpg")]
    all_images.append(images)   
for i in folders:
    readimg(i[0])

### pre-processing of Images

In [5]:
#importing the required packages
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage import io, color, img_as_ubyte

In [6]:
z=-1
a=[]
for i in all_images:
    z+=1
    p=[]
    for img in i:
        k=[]
        grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converts the image into grayscale mode
        #guassian filter is anlow pass filter used for reducing noise(high frequency components) and blurring regions of an image
        grayscale=cv2.GaussianBlur(grayscale,(7,7),0)
        histr = cv2.calcHist([grayscale],[0],None,[256],[0,256])
        otsu_threshold, image_result=cv2.threshold(grayscale,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY )
        image_result[image_result==255],image_result[image_result==0]=1,255
        image_result[image_result==1]=0
        kernel = np.ones((5,5),np.uint8)
        dilation = cv2.dilate(image_result,kernel,iterations = 1)
        erosion = cv2.erode(image_result,kernel,iterations = 1)
        closing = cv2.morphologyEx(image_result, cv2.MORPH_CLOSE, kernel)
        contours,hierarchy = cv2.findContours(dilation, 1, 2)
        cnt = contours[0]
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
        k.append(area)
        k.append(perimeter)
        b,g,r = cv2.split(img) 
        k.append(np.mean(r))
        k.append(np.mean(b))
        k.append(np.mean(g))
        k.append(np.std(r))
        k.append(np.std(b))
        k.append(np.std(g))
        bitand=[]
        bitand.append(cv2.bitwise_and(closing,r))
        bitand.append(cv2.bitwise_and(closing,g))
        bitand.append(cv2.bitwise_and(closing,b))
        image= cv2.merge([r,g,b])
        gray = color.rgb2gray(image)
        image = img_as_ubyte(gray)
        bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
        inds = np.digitize(image, bins)
        max_value = inds.max()+1
        matrix_coocurrence = greycomatrix(inds, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=False, symmetric=False)
        k+=greycoprops(matrix_coocurrence, 'contrast').tolist()[0]
        k+= greycoprops(matrix_coocurrence, 'dissimilarity').tolist()[0]
        k+= greycoprops(matrix_coocurrence, 'homogeneity').tolist()[0]
        k+= greycoprops(matrix_coocurrence, 'energy').tolist()[0]
        k+= greycoprops(matrix_coocurrence, 'correlation').tolist()[0]
        k+= greycoprops(matrix_coocurrence, 'ASM').tolist()[0]
        k.append(all_subdirs[z])
        p.append(k)
    a.append(p)


In [7]:
import csv
h=["area","perimeter","r_mean","b_mean","g_mean","r_std","b_std","g_std","contrast_1","contrast_2","contrast_3","contrast_4","dissimlarity_1","dissimlarity_2","dissimlarity_3","dissimlarity_4","homogeneity_1","homogeneity_2","homogeneity_3","homogeneity_4","energy_1","energy_2","energy_3","energy_4","correlation_1","correlation_2","correlation_3","correlation_4","asm_1","asm_2","asm_3","asm_4","class_level"]
with open(r'C:\Users\pavan\Downloads\leaf_data_set.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(h)

    # write multiple rows
    for i in range(len(a)):
        writer.writerows(a[i])

In [8]:
import pandas as pd
data=pd.read_csv(r'C:\Users\pavan\Downloads\leaf_data_set.csv')
data.head()

,area,perimeter,r_mean,b_mean,g_mean,r_std,b_std,g_std,contrast_1,contrast_2,...,energy_4,correlation_1,correlation_2,correlation_3,correlation_4,asm_1,asm_2,asm_3,asm_4,class_level
0,4.0,7.656854,113.009567,98.667679,124.975052,52.151884,62.930396,45.098714,4.404412,4.832449,...,0.132954,0.741145,0.715799,0.733520,0.725497,0.018605,0.017531,0.018137,0.017677,Apple___Apple_scab
1,2.0,5.656854,119.944580,110.333755,128.267365,54.634916,63.561602,45.730131,4.884544,5.526997,...,0.136741,0.731875,0.696846,0.716856,0.718881,0.019944,0.018362,0.019128,0.018698,Apple___Apple_scab
2,67.5,44.526911,128.512802,123.811050,128.722946,44.779738,59.958839,40.243447,4.249173,4.387190,...,0.145336,0.691007,0.681434,0.685975,0.689367,0.021931,0.021225,0.021812,0.021123,Apple___Apple_scab
3,315.0,92.142135,104.932266,105.313095,113.368484,57.670711,66.643183,50.720956,4.353784,4.785006,...,0.122645,0.798387,0.778590,0.787288,0.779954,0.015750,0.014900,0.015421,0.015042,Apple___Apple_scab
4,2.0,5.656854,121.570267,111.120163,125.261810,38.714286,48.490118,31.778066,3.799203,4.137978,...,0.182029,0.571445,0.533520,0.533314,0.533836,0.035068,0.033023,0.034162,0.033135,Apple___Apple_scab


In [9]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [10]:
x=data.iloc[:,0:32]
y=data.iloc[:,32]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier
tree= DecisionTreeClassifier(criterion="entropy", max_depth=10)

tree.fit(x_train, y_train)
predicted = tree.predict(x_test)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, predicted))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, predicted,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, predicted,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, predicted,average='macro'))

ACCURACY OF THE MODEL:  0.5697738554881412
F1 SCORE OF THE MODEL:  0.4569030747834686
RECALL OF THE MODEL:  0.45647726034180414
PRECISION OF THE MODEL:  0.4781476104322852


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
logmodel = LogisticRegression()
logmodel.fit(x_train,y_train)
predictions = logmodel.predict(x_test)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, predictions))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, predictions,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, predictions,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, predictions,average='macro'))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ACCURACY OF THE MODEL:  0.694980694980695
F1 SCORE OF THE MODEL:  0.6104074935034023
RECALL OF THE MODEL:  0.6007829795726386
PRECISION OF THE MODEL:  0.64382042359245


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 1000)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn import metrics
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, y_pred,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, y_pred,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, y_pred,average='macro'))

ACCURACY OF THE MODEL:  0.7192498621070049
F1 SCORE OF THE MODEL:  0.6179632002134836
RECALL OF THE MODEL:  0.6007878220056981
PRECISION OF THE MODEL:  0.7108349713858516


In [16]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(x_train, y_train) 
y_pred= classifier.predict(x_test)  
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, y_pred,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, y_pred,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, y_pred,average='macro'))

ACCURACY OF THE MODEL:  0.7079977937120794
F1 SCORE OF THE MODEL:  0.6091935495693668
RECALL OF THE MODEL:  0.6006450286384366
PRECISION OF THE MODEL:  0.6320501220658374


In [17]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state = 1)
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, y_pred,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, y_pred,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, y_pred,average='macro'))

ACCURACY OF THE MODEL:  0.7457253171538886
F1 SCORE OF THE MODEL:  0.6369564041577269
RECALL OF THE MODEL:  0.6220976737483972
PRECISION OF THE MODEL:  0.6967643754992434


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
y = data.select_dtypes(include=[object])
y.class_level.unique()
y.head()

,class_level
0,Apple___Apple_scab
1,Apple___Apple_scab
2,Apple___Apple_scab
3,Apple___Apple_scab
4,Apple___Apple_scab


In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = y.apply(le.fit_transform)

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [22]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
mlp.fit(x_train, y_train.values.ravel())
predictions = mlp.predict(x_test)


In [23]:
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, predictions))
print("F1 SCORE OF THE MODEL: ", metrics.f1_score(y_test, predictions,average='macro'))
print("RECALL OF THE MODEL: ", metrics.recall_score(y_test, predictions,average='macro'))
print("PRECISION OF THE MODEL: ", metrics.precision_score(y_test, predictions,average='macro'))

ACCURACY OF THE MODEL:  0.7485109199205824
F1 SCORE OF THE MODEL:  0.6357785294077583
RECALL OF THE MODEL:  0.6275324950758766
PRECISION OF THE MODEL:  0.6615499738471742
